![Model Training](https://drive.google.com/uc?id=17cUxpbL2eonLzgY0uIVAxmsfMfQF1by2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls '/content/drive/My Drive/Colab Notebooks/twitter.properties'

In [ ]:
!pip install ConfigParser

[twitter]
accesstoken=xxxx

accesstokensecret=xxxx

apikey=xxxxxx

apisecretkey=xxxxxx

googleapikey=xxxxxxx

In [ ]:
import configparser
config = configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/twitter.properties')

print(config.sections());


In [ ]:
import os
import tweepy as tw

In [ ]:
accesstoken=config.get('twitter', 'accesstoken')
accesstokensecret=config.get('twitter', 'accesstokensecret')
apikey=config.get('twitter', 'apikey')
apisecretkey=config.get('twitter', 'apisecretkey')

In [ ]:
auth = tw.OAuthHandler(apikey, apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
search_words = "#coronavirus"
date_since = "2020-03-10"

In [ ]:
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(1000)

In [ ]:
tweets

In [ ]:
tweet_details = [[tweet.geo, tweet.text,tweet.user.screen_name, tweet.user.location] for tweet in tweets]
#tweet_details

In [ ]:
import pandas as pd

In [ ]:
tweet_df = pd.DataFrame(data=tweet_details, columns=['geo','text','user', "location"])

In [ ]:
pd.set_option('max_colwidth', 800)

tweet_df.head(20)

In [ ]:
tweet_df.user.value_counts()

In [ ]:
tweet_df.location.value_counts()

In [ ]:
import re
def clean_tweets(text):
    text = re.sub("RT @[\w]*:","",text)
    text = re.sub("@[\w]*","",text)
    text = re.sub("https?://[A-Za-z0-9./]*","",text)
    text = re.sub("\n","",text)
    return text

In [ ]:
tweet_df['text']=tweet_df['text'].apply(lambda x: clean_tweets(x))

In [ ]:
tweet_df.head(20)

In [ ]:
tweet_df.to_csv('tweets.csv')

In [ ]:
!ls

In [ ]:
import spacy 
  
nlp = spacy.load('en_core_web_sm') 

In [ ]:
tweet_df['text'].apply(lambda x: [print("\tText : {}, Entity : {}".format(ent.text, ent.label_)) if (not ent.text.startswith('#')) else ""  for ent in nlp(x).ents])

In [ ]:
tweet_df['entities']=tweet_df['text'].apply(lambda x: [(ent.text, ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

In [ ]:
tweet_df.head(20)

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
tweet_df['sentiment']=tweet_df['text'].apply(lambda x: sid.polarity_scores(x))

In [ ]:
tweet_df.head(10)

In [ ]:
!pip install googlemaps

In [ ]:
import googlemaps

In [ ]:
gmaps = googlemaps.Client(key=config.get('twitter', 'googleapikey'))

geocode_result = gmaps.geocode(tweet_df['location'][1])

print(geocode_result)
print(geocode_result[0]['formatted_address'])

print(geocode_result[0]['formatted_address'].split(",")[-1].strip())



In [ ]:
def get_country(input):
  try:
    output=gmaps.geocode(input)[0]['formatted_address'].split(",")[-1].strip()
  except:
    output="Error"
  return output

In [ ]:
tweet_df['country']=tweet_df['location'].apply(lambda x: "" if (not x.strip()) else get_country(x))

In [ ]:
tweet_df['country'].value_counts()

In [ ]:
tweet_df.head(10)

In [ ]:
tweet_df.dtypes

In [ ]:
tweet_df.to_csv("tweets.csv")